# ECS713: week 6 Lab Sheet

This lab sheet covers 
- IO

## Learning Objectives

By the time you complete this sheet you should be able to
- understand how to declare a new typeclass
- how to make a type an instance of a typeclass
- recognise some common typeclasses and the functions they provide
- understand the use of the deriving connective

## Turn off the annoying linter

Run the cell below to turn off the annoying linter, which suggests improvements to your code that aren't appropriate for these exercises. 

In [1]:
:opt no-lint

## Task 1. Basic IO

**Problems with Jupyter** There are problems with using Jupyter notebooks in connection with IO. The basic issue is that standard Haskell IO functions are fairly simple-minded, and the framework you are likely to be using to run the notebooks is not. If you are doing what I am, then you are running the server on a Docker container. This means in particular that it has an encapsulated filesystem. So the starting point is that Haskell expects interactions to be taking place in the container. This is a particular issue for inputs. The notebook will look for files in the container's filesystem, not the one on your computer. This is not so bad, you can always upload things. But it is a real issue for keyboard input. It will try to take input from a handle called "stdin". This is not connected to your keyboard, so it will appear empty. Here is a simple experiment, run the cell below: 

In [2]:
getLine

"test"

If you are running this on our hub, then it should work just fine. You should get a box to enter a String into. Put in a string and hit enter. 

If you are running on Docker on your own laptop you may get something like: 

```<stdin>: hGetLine: end of file```

This basically says that the notebook is not capable of getting input in this form. The best way to get round that is to install Haskell (better install stack which we will be using later), and then use the interpreter: ghci. We are shortly going to move off Jupyter notebooks because we want to be able to access non-Haskell systems such as databases. 

We can give our input a name using the back arrow notation we saw in Lecture 1: 

In [ ]:
x <- getLine

In [ ]:
x

However, output is OK. Haskell sends its results on "stdout", so this is connected up appropriately, and you may not even be able to see the critical difference between the notebook printing out the result of a calculation, and something being printed as a side effect during a calculation. 

In [ ]:
-- String printed out as result
"Hi there!"

In [ ]:
-- String printed out during IO operation. 
putStrLn "Hi there!"

Basic IO is documented at: https://hackage.haskell.org/package/base-4.14.0.0/docs/Prelude.html#g:25

The simple functions can be found at: https://hackage.haskell.org/package/base-4.14.0.0/docs/Prelude.html#g:26

For the purpose of these exercises we will mainly be using file-based IO: https://hackage.haskell.org/package/base-4.14.0.0/docs/Prelude.html#g:29

In particular we will want: 

```type FilePath = String
readFile :: FilePath -> IO String
writeFile :: FilePath -> String -> IO ()```


We have a file "poem.txt", to access it: 

In [2]:
readFile "poem.txt"

"Twas brillig, and the slithy toves\nDid gyre and gimble in the wabe:\nAll mimsy were the borogoves,\nAnd the mome raths outgrabe.\n\n\"Beware the Jabberwock, my son!\nThe jaws that bite, the claws that catch!\nBeware the Jubjub bird, and shun\nThe frumious Bandersnatch!\"\n\nHe took his vorpal sword in hand;\nLong time the manxome foe he sought-\nSo rested he by the Tumtum tree\nAnd stood awhile in thought.\n\nAnd, as in uffish thought he stood,\nThe Jabberwock, with eyes of flame,\nCame whiffling through the tulgey wood,\nAnd burbled as it came!\n\nOne, two! One, two! And through and through\nThe vorpal blade went snicker-snack!\nHe left it dead, and with its head\nHe went galumphing back.\n\n\"And hast thou slain the Jabberwock?\nCome to my arms, my beamish boy!\nO frabjous day! Callooh! Callay!\"\nHe chortled in his joy.\n\n'Twas brillig, and the slithy toves\nDid gyre and gimble in the wabe:\nAll mimsy were the borogoves,\nAnd the mome raths outgrabe.\n      "

We can write definitions: 

In [ ]:
ioPoem = readFile "poem.txt"
poem <- readFile "poem.txt"
:t ioPoem
:t poem

The `<-` form only works in the context of a `do` block, which the Jupyter cells implicitly are. 

`ioPoem` is an `IO String`, not a `String`:

In [ ]:
length ioPoem

But `poem` is a `String`.

In [ ]:
length poem

You can do local declarations with IO, but you can't use the `<-` form: 

In [ ]:
let poem <- readFile "poem.txt" in length poem

Let's import the `Data.Char` module so we have a function to map characters to upper case: 

In [ ]:
import Data.Char

And use it as in lectures to map everything to upper case: 

In [ ]:
do 
  -- IO read stuff here
  poem <- readFile "poem.txt"
  -- Pure (non-IO) calculations here
  let upperPoem = map toUpper poem
  -- IO write (output) stuff here
  putStr upperPoem

Haskell is very uniform. We can assign this `do` block a name: 

In [ ]:
printUpperPoem = 
  do 
    -- IO read stuff here
    poem <- readFile "poem.txt"
    -- Pure (non-IO) calculations here
    let upperPoem = map toUpper poem
    -- IO write (output) stuff here
    putStr upperPoem

And then we can run it: 

In [ ]:
printUpperPoem

We can extract the filename "poem.txt" and make a function that will convert any file to upper case: 

In [ ]:
printUpper filename = 
  do 
    -- IO read stuff here
    poem <- readFile filename
    -- Pure (non-IO) calculations here
    let upperPoem = map toUpper poem
    -- IO write (output) stuff here
    putStr upperPoem

In [ ]:
:type printUpper

In [ ]:
printUpper "john.vcard"

## Task 2. Do block structure

Do blocks contain a sequence of declarations: 
- IO binders: ``<name> <- <io_expression>``
- ordinary declarations ``let <name> = <expression>`` (note the `let`)
- expressions

They must end with an expression of IO type. If necessary use the return keyword to achieve this. 

Try this with and without the `return`:

In [ ]:
do 
  poem <- readFile "poem.txt"
  let first_word = head $ words poem
  return first_word

Only the last `return` counts: 

In [ ]:
do 
  poem <- readFile "poem.txt"
  let first_word = head $ words poem
  return first_word
  let second_word = head $ tail $ words poem
  return second_word

There is a subtle point here. The do block above returns an `IO String` which is being executed and printed (we're returning a `String`). This on the other hand returns an `IO ()`, and just does the printing: 

In [ ]:
do 
  poem <- readFile "poem.txt"
  let first_word = head $ words poem
  putStrLn first_word
  let second_word = head $ tail $ words poem
  putStrLn second_word

**Exercise:** Write a simple word count program in the form of a function: 

```wc :: FilePath -> IO ()```

This should count the lines, words and characters of the relevant file, and print them with the name of the file. So 

``wc "poem.txt"`` 

should print: 

``poem.txt
lines: 35
words: 166
chars: 939``

Note: you may find the functions, `lines`, `words` and `length` helpful, as well as the function `show` which 
converts an element of a printable type into a String, so that you can print it out.  

In [5]:
wc :: FilePath -> IO ()
wc filename = do
   poem <- readFile filename
   putStrLn filename
   putStrLn $ "lines: " ++ show(length $ lines poem)
   putStrLn $ "words: " ++ show(length $ words poem)
   putStrLn $ "words: " ++ show(length poem)

In [6]:
wc "poem.txt"

poem.txt
lines: 35
words: 166
words: 939